In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from QBUS2820 import forward
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor,RandomForestRegressor
from sklearn import linear_model
from sklearn.neighbors import RadiusNeighborsRegressor,NearestNeighbors

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Using standardised data for LASSO, and Ridge

In [2]:
data = pd.read_csv('TrainStandard.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

y_train
mu=y_train.mean()
sigma=y_train.std() 

standardPrice=(y_train-mu)/sigma

In [3]:
method = []
pred = []

In [4]:
#LASSO
lasso = LassoCV(cv=10)
lasso.fit(final_train, np.ravel(standardPrice)) 
pred_L = lasso.predict(final_test)
predFinalLasso = (pred_L*sigma) + mu
method.append('LASSO')
pred.append(predFinalLasso)

In [5]:
#RIDGE
alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=10)
ridge.fit(final_train, np.ravel(standardPrice))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(standardPrice))
pred_R = ridge.predict(final_test)
predFinalRidge = (pred_R*sigma) + mu
method.append('Ridge')
pred.append(predFinalRidge)

In [6]:
#ENET
#Elastic Net

enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=10)
enet.fit(final_train, np.ravel(standardPrice))

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(final_train, np.ravel(standardPrice))
pred_E = enet.predict(final_test)
predFinalEnet = (pred_E*sigma) + mu
method.append('ENET')
pred.append(predFinalEnet)

In [8]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
LASSO,22302.821,3273.858,0.898,0.027,14657.405,0.898
Ridge,22198.874,3205.593,0.899,0.027,14814.092,0.899
ENET,22302.881,3274.824,0.898,0.027,14657.273,0.898


In [10]:
data = pd.read_csv('TrainSale1.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')


GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(predFinalBoost)

#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(predforward)




In [12]:
regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(adapred)


In [14]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='lad', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost LAD')
pred.append(predFinalBoost)

In [15]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
LASSO,22302.821,3273.858,0.898,0.027,14657.405,0.898
Ridge,22198.874,3205.593,0.899,0.027,14814.092,0.899
ENET,22302.881,3274.824,0.898,0.027,14657.273,0.898
GBoost,20998.196,3485.240,0.910,0.020,13910.467,0.910
Forward,23308.651,2581.009,0.889,0.022,15289.870,0.889
AdaBoost,25066.368,2520.651,0.872,0.016,18337.600,0.872
GBoost LAD,21317.049,3559.588,0.907,0.021,14102.206,0.907


### Predictions

In [ ]:
#Using standardised data for ridge and lasso
data = pd.read_csv('TrainStandard.csv')
y_train = data.pop('SalePrice')

y_train
mu=y_train.mean()
sigma=y_train.std() 

standardPrice=(y_train-mu)/sigma
kaggle1 = pd.read_csv('TestStandard.csv')

#RIDGE
alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=10)
ridge.fit(data, np.ravel(standardPrice))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(data, np.ravel(standardPrice))
pred_R = ridge.predict(kaggle1)
predFinalRidge = (pred_R*sigma) + mu


In [16]:
data = pd.read_csv('Train6.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6.csv')

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)

In [17]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction = pd.DataFrame({'Id':ind, 'Prediction':predFinalBoost})
prediction
#Saving results into CSV file 
prediction.to_csv("PredictionDay3_1.csv", index=False)

### Really strong performance

In [18]:
#Forward selection
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)


In [20]:
finalPred2 = (predforward+predFinalBoost)/2
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalPred2})
prediction2
#Saving results into CSV file 
prediction2.to_csv("PredictionDay3_2.csv", index=False)

### Slightly better result

In [21]:
#Extremely Random forest
regr = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr = regr.fit(data,y_price)
predFinalExtRandomForestlad = regr.predict(kaggle)


In [22]:
finalPred3 = (predFinalExtRandomForestlad+predFinalBoost+predforward)/3
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction3 = pd.DataFrame({'Id':ind, 'Prediction':finalPred3})
prediction3
#Saving results into CSV file 
prediction3.to_csv("PredictionDay3_3.csv", index=False)

### Result has improved again

In [23]:
#Using standardised data for ridge and lasso
data = pd.read_csv('TrainStandard.csv')
y_train = data.pop('SalePrice')

y_train
mu=y_train.mean()
sigma=y_train.std() 

standardPrice=(y_train-mu)/sigma
kaggle1 = pd.read_csv('TestStandard.csv')

#RIDGE
alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=10)
ridge.fit(data, np.ravel(standardPrice))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(data, np.ravel(standardPrice))
pred_R = ridge.predict(kaggle1)
predFinalRidge = (pred_R*sigma) + mu


In [24]:
finalPred4 = (predFinalExtRandomForestlad+predFinalBoost+predforward+predFinalRidge)/4
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction4 = pd.DataFrame({'Id':ind, 'Prediction':finalPred4})
prediction4
#Saving results into CSV file 
prediction4.to_csv("PredictionDay3_4.csv", index=False)

# Ridge Regression is bad for our final Model

In [35]:
data = pd.read_csv('Train6.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6.csv')

regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=11),random_state=0)
regr = regr.fit(data,y_price)
predAda = regr.predict(kaggle)

In [36]:
#Extremely Random forest
regr = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr = regr.fit(data,y_price)
predFinalExtRandomForestlad = regr.predict(kaggle)


In [37]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)

In [38]:
#Forward selection
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)


In [39]:
finalPred5 = (predFinalExtRandomForestlad+predFinalBoost+predforward+predAda)/4
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalPred5})
prediction5
#Saving results into CSV file 
prediction5.to_csv("PredictionDay3_5.csv", index=False)